In [1]:
import numpy as np
import pandas as pd
import requests
import json
import dotenv
import os

## Getting biographical data from the Congress API
Read the API documentation efficiently by looking only for these five things:
1. Root: the beginning of part of all URLs associated with this API for getting data back (look in the examples)
2. Endpoint: the second part of the URL, generally points to a specific data set to be returned  
3. Headers: data that you give to the API to identify yourself and provide other information if needed
    * User Agent String: a string that identifies your software - if you are writing a bot, it includes your bot's name, and what methods you are using to access the API.  
    Standard convention: 'botname/version (email) python-requests/request-package-version'
4. API Parameters: ways to control what data is requested, generally listed in the documentation for each endpoint. 3 kinds of parameters:
    * Query: listed in a dicitonary and passed to the `params` argument of `requests.get()`
    * Path: are part of the endpoint
    * Global: apply to any use of the API regardless of enpoint, often inludes your API key, can be in either the headers or the params
5. Key: if an API key is needed, determine how to get one and how you will let the API know the key.  
    API keys should be saved in a .env file in the same folder as your code, use the `dotenv` package to load the key into your code without ever exposing the key

In [2]:
bioguideId = 'M001239'
root = 'https://api.congress.gov/v3'
endpoint = f'/member/{bioguideId}'

In [3]:
botname = 'ds6600'
version = '1.0'
email = 'jkropko@virginia.edu'
useragent = f'{botname}/{version} ({email}) python-requests/{requests.__version__}'
headers = {'User-Agent': useragent}
headers

{'User-Agent': 'ds6600/1.0 (jkropko@virginia.edu) python-requests/2.32.5'}

In [4]:
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
params = {'format': 'json', 'api_key': congresskey}

In [5]:
r = requests.get(root + endpoint, headers=headers, params=params,)
r.json()

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'officeAddress': '1013 Longworth House Office Building',
   'phoneNumber': '(202) 225-4711',
   'zipCode': 20515},
  'bioguideId': 'M001239',
  'birthYear': '1968',
  'cosponsoredLegislation': {'count': 170,
   'url': 'https://api.congress.gov/v3/member/M001239/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': 'Image courtesy of the Member',
   'imageUrl': 'https://www.congress.gov/img/member/67744ba20b34857ecc909149_200.jpg'},
  'directOrderName': 'John J. McGuire III',
  'district': 5,
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'McGuire, John J.',
  'lastName': 'McGuire',
  'middleName': 'J.',
  'officialWebsiteUrl': 'https://mcguire.house.gov/',
  'partyHistory': [{'partyAbbreviation': 'R',
    'partyName': 'Republican',
    'startYear': 2025}],
  'previousNames': [{'directOrderName': 'John McGuire',
    'endDate': '2025-01-10T17:36:18Z',
   

## Sponsored legislation

In [6]:
endpoint = f'/member/{bioguideId}/sponsored-legislation'
params = {'format': 'json', 'api_key': congresskey, 'offset': 0, 'limit': 250}

In [7]:
r = requests.get(root + endpoint, headers=headers, params=params,)
myjson = r.json()
myjson

{'pagination': {'count': 9},
 'request': {'bioguideId': 'm001239',
  'contentType': 'application/json',
  'format': 'json'},
 'sponsoredLegislation': [{'congress': 119,
   'introducedDate': '2025-09-26',
   'latestAction': {'actionDate': '2025-09-26',
    'text': 'Referred to the House Committee on Ways and Means.'},
   'number': '5595',
   'policyArea': {'name': None},
   'title': 'Requiring Excise for Migrant Income Transfers Act” or the “REMIT Act.',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5595?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-19',
   'latestAction': {'actionDate': '2025-09-19',
    'text': 'Referred to the House Committee on Armed Services.'},
   'number': '5522',
   'policyArea': {'name': None},
   'title': 'Peace Through Strength Act of 2025',
   'type': 'HR',
   'url': 'https://api.congress.gov/v3/bill/119/hr/5522?format=json'},
  {'congress': 119,
   'introducedDate': '2025-09-09',
   'latestAction': {'actionDate': '20

In [8]:
sponsoredlegislation_df = pd.json_normalize(myjson, record_path=['sponsoredLegislation'])
sponsoredlegislation_df

,congress,introducedDate,number,title,type,url,latestAction.actionDate,latestAction.text,policyArea.name
0,119,2025-09-26,5595,Requiring Excise for Migrant Income Transfers ...,HR,https://api.congress.gov/v3/bill/119/hr/5595?f...,2025-09-26,Referred to the House Committee on Ways and Me...,None
1,119,2025-09-19,5522,Peace Through Strength Act of 2025,HR,https://api.congress.gov/v3/bill/119/hr/5522?f...,2025-09-19,Referred to the House Committee on Armed Servi...,None
2,119,2025-09-09,5218,Border Wall Status Act,HR,https://api.congress.gov/v3/bill/119/hr/5218?f...,2025-09-09,Referred to the House Committee on Homeland Se...,Immigration
3,119,2025-09-03,5103,Make the District of Columbia Safe and Beautif...,HR,https://api.congress.gov/v3/bill/119/hr/5103?f...,2025-09-10,Ordered to be Reported (Amended) by the Yeas a...,Public Lands and Natural Resources
4,119,2025-07-22,4603,FAIR Act,HR,https://api.congress.gov/v3/bill/119/hr/4603?f...,2025-07-22,Referred to the House Committee on Energy and ...,Energy
5,119,2025-05-29,3643,VA Data Transparency and Trust Act,HR,https://api.congress.gov/v3/bill/119/hr/3643?f...,2025-06-12,Subcommittee Hearings Held,Armed Forces and National Security
6,119,2025-05-17,3478,Manned Aircraft Clarification Act,HR,https://api.congress.gov/v3/bill/119/hr/3478?f...,2025-05-18,Referred to the Subcommittee on Aviation.,Crime and Law Enforcement
7,119,2025-02-26,1622,Uranium for Energy Independence Act of 2025,HR,https://api.congress.gov/v3/bill/119/hr/1622?f...,2025-02-26,Referred to the House Committee on Natural Res...,Energy
8,119,2025-02-21,1487,Agricultural and Forestry Hauling Efficiency Act,HR,https://api.congress.gov/v3/bill/119/hr/1487?f...,2025-02-26,Sponsor introductory remarks on measure. (CR H...,Transportation and Public Works


In [9]:
for x in sponsoredlegislation_df['title']:
    print(x)

Requiring Excise for Migrant Income Transfers Act” or the “REMIT Act.
Peace Through Strength Act of 2025
Border Wall Status Act
Make the District of Columbia Safe and Beautiful Act
FAIR Act
VA Data Transparency and Trust Act
Manned Aircraft Clarification Act
Uranium for Energy Independence Act of 2025
Agricultural and Forestry Hauling Efficiency Act


In [10]:
r = requests.get(sponsoredlegislation_df['url'][3], headers=headers, params=params)
r.json()

{'bill': {'actions': {'count': 6,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/actions?format=json'},
  'cboCostEstimates': [{'description': 'As ordered reported by the House Committee on Oversight and Government Reform on \nSeptember 10, 2025\n',
    'pubDate': '2025-10-03T14:03:00Z',
    'title': 'H.R. 5103, Make the District of Columbia Safe and Beautiful Act of 2025',
    'url': 'https://www.cbo.gov/publication/61792'}],
  'committees': {'count': 2,
   'url': 'https://api.congress.gov/v3/bill/119/hr/5103/committees?format=json'},
  'congress': 119,
  'constitutionalAuthorityStatementText': '<pre>\n[Congressional Record Volume 171, Number 144 (Wednesday, September 3, 2025)]\n[House]\nFrom the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]\nBy Mr. McGUIRE:\nH.R. 5103.\nCongress has the power to enact this legislation pursuant\nto the following:\nArticle I, Section 8\n[Page H3829]\n</pre>',
  'cosponsors'

## FEC data for financial campaign contributions

In [11]:
dotenv.load_dotenv()
feckey = os.getenv('feckey')

In [12]:
root = 'https://api.open.fec.gov'
endpoint = '/v1/candidates/search'
params = {'api_key': feckey, 'q': 'John McGuire', 'state': 'VA', 'district': '5', 'office': 'H'}

In [13]:
r = requests.get(root + endpoint, headers=headers, params=params)
r.json()

{'api_version': '1.0',
 'pagination': {'count': 1,
  'is_count_exact': True,
  'page': 1,
  'pages': 1,
  'per_page': 20},
 'results': [{'active_through': 2026,
   'candidate_id': 'H0VA07133',
   'candidate_inactive': False,
   'candidate_status': 'C',
   'cycles': [2020, 2026],
   'district': '05',
   'district_number': 5,
   'election_districts': ['07', '05'],
   'election_years': [2020, 2026],
   'federal_funds_flag': False,
   'first_file_date': '2019-11-18',
   'has_raised_funds': True,
   'inactive_election_years': None,
   'incumbent_challenge': 'C',
   'incumbent_challenge_full': 'Challenger',
   'last_f2_date': '2024-11-11',
   'last_file_date': '2024-11-11',
   'load_date': '2025-02-02T20:58:51',
   'name': 'MCGUIRE, JOHN',
   'office': 'H',
   'office_full': 'House',
   'party': 'REP',
   'party_full': 'REPUBLICAN PARTY',
   'principal_committees': [{'affiliated_committee_name': 'MCGUIRE VICTORY FUND',
     'candidate_ids': ['H0VA07133'],
     'committee_id': 'C00856831',
  

In [14]:
myjson = r.json()
candidate_id = myjson['results'][0]['candidate_id']
candidate_id

'H0VA07133'

In [27]:
endpoint = f'/v1/candidate/{candidate_id}/committees/'
params = {'api_key': feckey, 'cycle': '2024'}

r = requests.get(root + endpoint, headers=headers, params=params)
r.json()['results']

[{'affiliated_committee_name': 'MCGUIRE VICTORY FUND',
  'candidate_ids': ['H0VA07133'],
  'city': 'MANAKIN SABOT',
  'committee_id': 'C00856831',
  'committee_type': 'H',
  'committee_type_full': 'House',
  'custodian_city': 'WASHINGTON',
  'custodian_name_1': 'JASON',
  'custodian_name_2': 'BOLES',
  'custodian_name_full': 'BOLES, JASON D',
  'custodian_name_middle': 'D',
  'custodian_name_prefix': None,
  'custodian_name_suffix': None,
  'custodian_name_title': 'TREASURER',
  'custodian_phone': '2025457706',
  'custodian_state': 'DC',
  'custodian_street_1': '126 C STREET NW',
  'custodian_street_2': 'THIRD FLOOR',
  'custodian_zip': '20001',
  'cycles': [2024, 2026],
  'designation': 'P',
  'designation_full': 'Principal campaign committee',
  'email': 'JASON@TABULARIUS.PRO',
  'fax': None,
  'filing_frequency': 'Q',
  'first_f1_date': '2023-11-15',
  'first_file_date': '2023-11-15',
  'form_type': 'F1',
  'last_f1_date': '2025-02-25',
  'last_file_date': '2025-09-09',
  'leadershi

In [28]:
[i['name'] for i in r.json()['results']]

['MCGUIRE FOR VIRGINIA', 'MCGUIRE RECOUNT COMMITTEE']

In [29]:
committee_ids = [i['committee_id'] for i in r.json()['results']]
committee_ids

['C00856831', 'C00900118']

In [36]:
endpoint = f'/v1/schedules/schedule_a'
params = {'api_key': feckey, 'committee_id': committee_ids[0], 'per_page': 100, 'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint, headers=headers, params=params)
r.json()

{'api_version': '1.0',
 'pagination': {'count': 1771,
  'is_count_exact': True,
  'last_indexes': {'last_contribution_receipt_amount': '5000.00',
   'last_index': '4032120251180734842'},
  'pages': 18,
  'per_page': 100},
 'results': [{'amendment_indicator': 'N',
   'amendment_indicator_desc': 'NO CHANGE',
   'back_reference_schedule_name': None,
   'back_reference_transaction_id': None,
   'candidate_first_name': None,
   'candidate_id': None,
   'candidate_last_name': None,
   'candidate_middle_name': None,
   'candidate_name': None,
   'candidate_office': None,
   'candidate_office_district': None,
   'candidate_office_full': None,
   'candidate_office_state': None,
   'candidate_office_state_full': None,
   'candidate_prefix': None,
   'candidate_suffix': None,
   'committee': {'affiliated_committee_name': 'MCGUIRE VICTORY FUND',
    'candidate_ids': ['H0VA07133'],
    'city': 'MANAKIN SABOT',
    'committee_id': 'C00856831',
    'committee_type': 'H',
    'committee_type_full': 'H

In [33]:
contrib_dict = [{'contributor_name': i['contributor_name'],
                'contributor_aggregate_ytd': i['contributor_aggregate_ytd'],
                'memo_text': i['memo_text'],
                'pdf_url': i['pdf_url']} for i in r.json()['results']]

contrib_df = pd.DataFrame(contrib_dict)
contrib_df.head(3)

,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...


In [37]:
lastindex = r.json()['pagination']['last_indexes']
lastindex

{'last_contribution_receipt_amount': '5000.00',
 'last_index': '4032120251180734842'}

In [38]:
endpoint = f'/v1/schedules/schedule_a'
params = {'api_key': feckey, 'committee_id': committee_ids[0], 'per_page': 100, 'sort': '-contribution_receipt_amount', 'last_contribution_receipt_amount': lastindex['last_contribution_receipt_amount'],'last_index': lastindex['last_index']}

r = requests.get(root + endpoint, headers=headers, params=params)
r.json()

{'api_version': '1.0',
 'pagination': {'count': 1771,
  'is_count_exact': True,
  'last_indexes': {'last_contribution_receipt_amount': '3300.00',
   'last_index': '4081520242013129312'},
  'pages': 18,
  'per_page': 100},
 'results': [{'amendment_indicator': 'A',
   'amendment_indicator_desc': 'ADD',
   'back_reference_schedule_name': None,
   'back_reference_transaction_id': None,
   'candidate_first_name': None,
   'candidate_id': None,
   'candidate_last_name': None,
   'candidate_middle_name': None,
   'candidate_name': None,
   'candidate_office': None,
   'candidate_office_district': None,
   'candidate_office_full': None,
   'candidate_office_state': None,
   'candidate_office_state_full': None,
   'candidate_prefix': None,
   'candidate_suffix': None,
   'committee': {'affiliated_committee_name': 'MCGUIRE VICTORY FUND',
    'candidate_ids': ['H0VA07133'],
    'city': 'MANAKIN SABOT',
    'committee_id': 'C00856831',
    'committee_type': 'H',
    'committee_type_full': 'House',

In [39]:
contrib_dict = [{'contributor_name': i['contributor_name'],
                'contributor_aggregate_ytd': i['contributor_aggregate_ytd'],
                'memo_text': i['memo_text'],
                'pdf_url': i['pdf_url']} for i in r.json()['results']]

contrib_df = pd.DataFrame(contrib_dict)
contrib_df.head(3)

,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,NATIONAL ASSOCIATION OF REALTORS PAC,5000.0,None,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1,CHEVRON EMPLOYEES PAC - CHEVRON CORPORATION,5000.0,None,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
2,AMERICAN ISRAEL PUBLIC AFFAIRS COMMITTEE PAC,5000.0,None,https://docquery.fec.gov/cgi-bin/fecimg/?20250...


In [40]:
endpoint = f'/v1/schedules/schedule_a'
params = {'api_key': feckey, 'committee_id': committee_ids[0], 'per_page': 100, 'sort': '-contribution_receipt_amount'}

r = requests.get(root + endpoint, headers=headers, params=params)
r.json()

contrib_list = [{'contributor_name': i['contributor_name'],
                'contributor_aggregate_ytd': i['contributor_aggregate_ytd'],
                'memo_text': i['memo_text'],
                'pdf_url': i['pdf_url']} for i in r.json()['results']]

lastindex = r.json()['pagination']['last_indexes']
newrecords = len(r.json()['results'])

while newrecords > 0:
    print(len(contrib_list))

    params['last_contribution_receipt_amount'] = lastindex['last_contribution_receipt_amount']
    params['last_index'] = lastindex['last_index']
    r = requests.get(root + endpoint, headers=headers, params=params)

    contrib_list += [{'contributor_name': i['contributor_name'],
                'contributor_aggregate_ytd': i['contributor_aggregate_ytd'],
                'memo_text': i['memo_text'],
                'pdf_url': i['pdf_url']} for i in r.json()['results']]
    
    lastindex = r.json()['pagination']['last_indexes']
    newrecords = len(r.json()['results'])

contrib_df = pd.DataFrame(contrib_list)
contrib_df

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1771


,contributor_name,contributor_aggregate_ytd,memo_text,pdf_url
0,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
1,MCGUIRE VICTORY FUND,119484.10,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
2,MCGUIRE VICTORY FUND,397795.08,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
3,MCGUIRE VICTORY FUND,66488.21,TRANSFER FROM JOINT FUNDRAISER,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
4,MCGUIRE VICTORY FUND,279791.09,None,https://docquery.fec.gov/cgi-bin/fecimg/?20240...
...,...,...,...,...
1766,"REHNERT, GEOFFREY",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1767,"GOOD, JOHN P MR",7000.00,REDESIGNATION FROM,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1768,"PEROT, ROSS",7000.00,MEMO: SPLIT OF-ROSS PEROT,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
1769,"MAFRIGE, DAVID Z.",6500.00,MEMO: SPLIT OF-DAVID Z. MAFRIGE,https://docquery.fec.gov/cgi-bin/fecimg/?20250...
